In [1]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import pandas as pd

In [53]:
# 이미지 로드
img = cv2.imread('image/erase/img_16.png')
info=pd.read_csv('image/information/gt_img_16.csv',encoding='cp949') 

## 흰색 배경에 검정 글씨 (검정 글씨 추출) 

In [50]:
# 이미지 읽기
text_img = cv2.imread('image/text/black.png')

# BGR에서 HSV로 변환
hsv_image = cv2.cvtColor(text_img, cv2.COLOR_BGR2HSV)

# 검정색 범위 설정 (HSV)
lower_black = np.array([0, 0, 0])
upper_black = np.array([179, 255, 30])

# 검정색 영역 추출
mask = cv2.inRange(hsv_image, lower_black, upper_black)

# 텍스트의 색상 추출 (평균값 계산)
b, g, r = cv2.mean(text_img, mask=mask)[:3]
text_color = (round(b), round(g), round(r))

## 검정 배경에 흰색 글씨 (흰색 글씨 추출)

In [57]:
text_img = cv2.imread('image/text/white.png')

# BGR에서 HSV로 변환
hsv_image = cv2.cvtColor(text_img, cv2.COLOR_BGR2HSV)

# 흰색 범위 설정 (HSV)
lower_white = np.array([0, 0, 200])
upper_white = np.array([179, 30, 255])

# 흰색 영역 추출
mask = cv2.inRange(hsv_image, lower_white, upper_white)

# 텍스트의 색상 추출 (평균값 계산)
b, g, r = cv2.mean(text_img, mask=mask)[:3]
text_color = (round(10000 * b), round(10000 * g), round(10000 * r))

# 실행 코드

In [58]:
#폰트 크기 및 사이즈 정의 함수
def add_text_to_image(img, text_list, font_path, font_sizes, text_color, positions):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    for i, (text, font_size, position) in enumerate(zip(text_list, font_sizes, positions)):
        font = ImageFont.truetype(font_path, font_size)
        text_x, text_y = position

        text_size = font.getsize(text)
        text_width, text_height = text_size

        draw.text((text_x, text_y), text, font=font, stroke_fill=text_color, fill=text_color)

    numpy_img = np.array(img_pil)
    cv_img = cv2.cvtColor(numpy_img, cv2.COLOR_RGB2BGR)
    return cv_img

In [59]:
# 좌표값 개수 입력 받기
#info=pd.concat([info,kor],axis=1)
num_coords = len(info) #5

# 좌표값과 텍스트 입력 받기
positions = []
text_list = []
font_sizes = []
for i in range(num_coords):
    if pd.notna(info.iloc[i][9]):
        bbox_start_x, bbox_start_y, bbox_end_x, bbox_end_y = int(info.iloc[i][0]), int(info.iloc[i][1]), int(info.iloc[i][4]), int(info.iloc[i][5])

        font_path = "image/font/malgun.ttf"

        # 높이 계산
        height = bbox_end_y - bbox_start_y
        font_size = int(height * 0.8)

        font = ImageFont.truetype(font_path, font_size)
        text_width, text_height = font.getsize(info.iloc[i][9])

        width = bbox_end_x - bbox_start_x
        x = bbox_start_x + width // 2 - text_width // 2  # bbox 내 가운데 정렬
        y = bbox_start_y + height // 2 - text_height // 2

        position = (x, y)
        text = info.iloc[i][9]
        positions.append(position)
        text_list.append(text)
        font_sizes.append(font_size)

# 이미지에 텍스트 추가
result_img = add_text_to_image(img, text_list, font_path, font_sizes, text_color, positions)

# 이미지 출력 및 저장
cv2.imshow('Results', result_img)
cv2.imwrite("image/results/16.png", result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\milks\AppData\Local\Temp\ipykernel_19752\2501440310.py:20: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width, text_height = font.getsize(info.iloc[i][9])
C:\Users\milks\AppData\Local\Temp\ipykernel_19752\910720760.py:10: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_size = font.getsize(text)
